<a href="https://colab.research.google.com/github/MeherPreetham/Genetic-Algorithm/blob/main/MonolithicGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install deap
import random
import math
from deap import base, creator, tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 2.8 MB/s eta 0:00:00


In [3]:
print("===========GA PARAMETERS===========")
NUM_TASKS = int(input("Enter the number of tasks: "))
NUM_CORES = int(input("Enter the number of cores: "))
NUM_POPULATION = int(input("Enter the number of population: "))
NUM_GENERATIONS = int(input("Enter the number of generations: "))
CROSSOVER_RATE = float(input("Enter the crossover rate: "))
MUTATION_RATE = float(input("Enter the mutation rate: "))
BASE_ENERGY = float(input("Enter the base energy consumption of the cores (J/ms): "))

===========GA PARAMETERS===========
Enter the number of tasks: 1000
Enter the number of cores: 16
Enter the number of population: 100
Enter the number of generations: 2000
Enter the crossover rate: 0.5
Enter the mutation rate: 0.2
Enter the base energy consumption of the cores (J/ms): 3


In [4]:
execution_times = [random.randint(1, 10) for _ in range(NUM_TASKS)]

def setup_deap(NUM_TASKS, NUM_CORES):
  creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
  creator.create("Individual", list, fitness = creator.FitnessMin)

  toolbox = base.Toolbox()

  toolbox.register("attr_core", random.randint, 0, NUM_CORES-1)
  toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_core, n = NUM_TASKS)
  toolbox.register("population", tools.initRepeat, list, toolbox.individual)

  return toolbox

toolbox = setup_deap(NUM_TASKS, NUM_CORES)
chromosome = toolbox.individual()

print("==============GENERATING AN INDIVIDUAL==============")
print("Sample task-to-core mapping (first 10 tasks):")
print(chromosome[:min(10, NUM_TASKS)])
print("\n")
for i, core in enumerate(chromosome[:min(10, NUM_TASKS)]):
  print(f"Task {i} → Core {core} | Execution Time: {execution_times[i]} ms")
print("====================================================")

==============GENERATING AN INDIVIDUAL==============
Sample task-to-core mapping (first 10 tasks):
[7, 4, 10, 12, 1, 9, 10, 3, 9, 10]


Task 0 → Core 7 | Execution Time: 4 ms
Task 1 → Core 4 | Execution Time: 9 ms
Task 2 → Core 10 | Execution Time: 8 ms
Task 3 → Core 12 | Execution Time: 5 ms
Task 4 → Core 1 | Execution Time: 3 ms
Task 5 → Core 9 | Execution Time: 1 ms
Task 6 → Core 10 | Execution Time: 8 ms
Task 7 → Core 3 | Execution Time: 10 ms
Task 8 → Core 9 | Execution Time: 9 ms
Task 9 → Core 10 | Execution Time: 2 ms


In [16]:
#Fitness

#To find the makespan; maximum time taken by a core to finish execution.
core_times = [0] * NUM_CORES

for task_id, core_id in enumerate(chromosome):
    exec_time = execution_times[task_id]
    core_times[core_id] += exec_time

makespan = max(core_times) #should be minimised.

#Logging and saving energy consumed by cores in total and per core.
power_cost = [0.0] * NUM_CORES #energy consumed by cores separately.

for task_id, core_id in enumerate(chromosome):
    exec_time = execution_times[task_id]
    power_cost[core_id] += exec_time * BASE_ENERGY

net_power_cost = sum(power_cost) #total energy consumed by all cores

#Calculating imbalance; task distribution across all the cores.
avg_load = sum(core_times) / NUM_CORES
#measures how off the load is from the average on all the cores separately.
imbalance = sum((time - avg_load) ** 2 for time in core_times) / NUM_CORES
MAX_EXEC_TIME = max(execution_times)
MAX_TOTAL = NUM_TASKS * MAX_EXEC_TIME
MAX_IMBALANCE = (MAX_TOTAL ** 2) / NUM_CORES


#Normalising the metrics between [0, 1]
norm_makespan = (makespan - min(core_times)) / (max(core_times) - min(core_times))
norm_energy = (net_power_cost - min(power_cost)) / (max(power_cost) - min(power_cost) + 1e-9)
norm_imbalance = imbalance / MAX_IMBALANCE

#The final weighted fitness score of all the constraints considered.
w1 = 0.5  # weight for makespan
w2 = 0.3  # weight for energy
w3 = 0.2  # weight for imbalance
fitness = 100 - ((w1 * norm_makespan) + (w2 * norm_energy) + (w3 * norm_imbalance))

print("==============EXECUTION TIME METRICS==============")
print(f"core times = {core_times}")
print(f"average core time = {avg_load}")
print(f"imbalance = {imbalance}")
print(f"normalised imbalance = {norm_imbalance}")
print(f"makespan = {makespan}")
print(f"normalised makespan = {norm_makespan}")
print("\n==================ENERGY METRICS==================")
print(f"power cost = {power_cost}")
print(f"net_power_cost = {net_power_cost}")
print(f"normalised energy = {norm_energy}")
print("\n=================WEIGHTED FITNESS=================")
print(f"Fitness = {fitness:.3f}%")
print("==================================================")

==============EXECUTION TIME METRICS==============
core times = [294, 381, 275, 371, 422, 338, 317, 286, 406, 298, 341, 335, 351, 381, 390, 308]
average core time = 343.375
imbalance = 1902.859375
normalised imbalance = 0.0003044575
makespan = 422
normalised makespan = 1.0

==================ENERGY METRICS==================
power cost = [882.0, 1143.0, 825.0, 1113.0, 1266.0, 1014.0, 951.0, 858.0, 1218.0, 894.0, 1023.0, 1005.0, 1053.0, 1143.0, 1170.0, 924.0]
net_power_cost = 16482.0
normalised energy = 35.50340136046371

=================WEIGHTED FITNESS=================
Fitness = 88.849%
